# 02450 Project 2


The primary goal of Project 2 is to apply supervised learning techniques (regression and classification) to predict properties or classifications of wood based on the cleaned and scaled dataset from Project 1.

In [136]:
import os
import numpy as np
import glob
import PIL.Image as Image
from tqdm.notebook import tqdm
import xlrd
import scipy
from scipy.linalg import svd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    plot,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from sklearn.naive_bayes import BernoulliNB  # Use BernoulliNB for binary classification
from sklearn.model_selection import KFold
import sklearn.linear_model as lm
from matplotlib.pylab import figure, hist, plot, show, subplot, xlabel, ylabel
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from dtuimldmtools import *
from dtuimldmtools.statistics.statistics import correlated_ttest
from dtuimldmtools import confmatplot, rocplot
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Cleaned and Scaled Data from Project 1

Hardwood= 1, Softwood = 0

In [137]:
cwd = os.getcwd()
data = pd.read_csv(cwd + '/usa_wood_data_formatted.csv')

removed_columns = ['side_hardness','tension_perpendicular_to_grain','impact_bending']
data_cleaned = data.drop(columns=removed_columns)
data_cleaned = data_cleaned.dropna()
print(data_cleaned.isnull().sum()) # 0 means data is clean

data_arr = np.array(data_cleaned)  # Attributes are columns
attributeNames = [name for name in data_cleaned.columns]
attributeNames = attributeNames[7:12]
attributeNames = np.concatenate((['Wood Type', 'Moisture Content'], attributeNames))
print(attributeNames)

for i in range(data_arr.shape[1]):
    val, count = np.unique(data_arr[:,i], return_counts=True)
    #print(val.shape)        # Print how many unique values of each attribute exist 
                            # Probably dont do one of k coding for first 4 attributes since they have so many unique values

data_adj = data_arr[:,6:13]
print("1",data_adj.shape)

for i in range(2):
    OoK = np.zeros([data_arr.shape[0], 1])
    val, count = np.unique(data_arr[:,5 - i], return_counts=True)
    
    for j in range(data_arr.shape[0]):
        if data_arr[j,5-i] == val[0]:
            OoK[j, 0] = 1
    data_adj = np.concatenate((OoK, data_adj), 1)
print("2",data_adj.shape)


#classsification
y = data_adj[:,0]
X = data_adj[:,1:9]
#X = np.concatenate(data_adj[:,2:9], 1)
#regression
#y = data_adj[:,4]
# = np.concatenate((data_adj[:,0:3], data_adj[:,5:]), 1)
N, M = X.shape

X = X.astype(np.float32)  # Convert to float32 (or float64 if necessary)
y = y.astype(np.float32)  # Convert y to float32

#print(y)
print("X-shape", X.shape)


# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)  # Features as float32
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Target variable as float32 and reshaped to (N, 1)
#X_tensor = torch.FloatTensor(X)
#y_tensor = torch.FloatTensor(X_scaled[:,6])
#y_tensor = torch.FloatTensor(y).unsqueeze(1)


common_species_name                   0
genus                                 0
species                               0
scientific_name                       0
classification                        0
moisture_content                      0
specific_gravity                      0
modulus_of_rupture                    0
modulus_of_elasticity                 0
work_to_maximum_load                  0
compression_parallel_to_grain         0
compression_perpendicular_to_grain    0
shear_parallel_to_grain               0
dtype: int64
['Wood Type' 'Moisture Content' 'modulus_of_rupture'
 'modulus_of_elasticity' 'work_to_maximum_load'
 'compression_parallel_to_grain' 'compression_perpendicular_to_grain']
1 (214, 7)
2 (214, 9)
X-shape (214, 8)


###  Two-Level Cross-Validation Model

K1 = outer loop

K2 = inner loop

h = no. of hidden neurons



Tasks for Classification:

Logistic regression

Method 2
2) ANN
3) Classification tree
4) k-nearest neighbor classification
5) Na¨ıve Bayes

Baseline:
Compute largest class on the training data, and predict everything in the test-data as belonging to that class (corresponding to the optimal prediction by a logistic regression model with a bias
term and no features)

In [138]:
def logistic_regression(X_train,y_train,X_test,y_test,mode):
    if mode == "return loss":
        X_train_mean = np.mean(X_train, axis=0)
        X_train_std = np.std(X_train, axis=0)
        # X_train_mean = np.expand_dims(X_train_mean, axis=1)
        # X_train_std = np.expand_dims(X_train_std, axis=1)

        for i in range(X_train.shape[0]):
            X_train[i] = (X_train[i, :] - X_train_mean) / X_train_std
        for i in range(X_test.shape[0]):
            X_test[i] = (X_test[i, :] - X_train_mean) / X_train_std

        logreg = lm.LogisticRegression(solver='lbfgs', multi_class='multinomial', tol=1e-4, random_state=1, max_iter=10000,penalty='l2', C=1/l)
        logreg.fit(X_train,y_train)
        return np.sum(logreg.predict(X_test)!=y_test)/len(y_test)
    elif mode == "return predictions":
        X_train_mean = np.mean(X_train, axis=0)
        X_train_std = np.std(X_train, axis=0)
        # X_train_mean = np.expand_dims(X_train_mean, axis=1)
        # X_train_std = np.expand_dims(X_train_std, axis=1)

        for i in range(X_train.shape[0]):
            X_train[i] = (X_train[i, :] - X_train_mean) / X_train_std
        for i in range(X_test.shape[0]):
            X_test[i] = (X_test[i, :] - X_train_mean) / X_train_std

        logreg = lm.LogisticRegression(solver='lbfgs', multi_class='multinomial', tol=1e-4, random_state=1, max_iter=10000,penalty='l2', C=1/l)
        logreg.fit(X_train,y_train)
        return logreg.predict(X_test)
        
logistic_regression(X_train,y_train,X_test,y_test,mode)

NameError: name 'mode' is not defined

In [ ]:
#Logistic regression model: Evaluating classification 

# Create crossvalidation partition for evaluation
K = 10
CV = model_selection.KFold(n_splits=K,shuffle=True)
#def logistic_regression(X_train,y_train,X_test,y_test,l,mode):

# Standardize the training and test set based on training set mean and std
mu = np.mean(X_train, 0)
sigma = np.std(X_train, 0)

X_train = (X_train - mu) / sigma
X_test = (X_test - mu) / sigma
print("mu (mean of X_train):", mu)
print("sigma (std of X_train):", sigma)

# Fit regularized logistic regression model to training data to predict the type of wine
#lambda_interval = np.logspace(-8, 2, 50)
lambda_interval = np.logspace(-2, 0, 50)  # From 10^-2 to 10^0 (0.01 to 1) with 50 points
#lambda_interval = np.power(10.0, range(-5, 9)) # Values of lambda from Part. XX write about this
# Check lambda values and intervals
#print("Lambda interval:", lambda_interval)

train_error_rate = np.zeros(len(lambda_interval))
test_error_rate = np.zeros(len(lambda_interval))
coefficient_norm = np.zeros(len(lambda_interval))

for k in range(0, len(lambda_interval)):
    mdl = LogisticRegression(penalty="l2", C=1 / lambda_interval[k]) #what exactly does Max Iter do?!

    mdl.fit(X_train, y_train)

    y_train_est = mdl.predict(X_train).T
    y_test_est = mdl.predict(X_test).T

    train_error_rate[k] = np.sum(y_train_est != y_train) / len(y_train)
    test_error_rate[k] = np.sum(y_test_est != y_test) / len(y_test)

    w_est = mdl.coef_[0]
    coefficient_norm[k] = np.sqrt(np.sum(w_est**2))
    print(f"\nLambda = {lambda_interval[k]} (C = {1 / lambda_interval[k]}):")
    print("Train Error Rate:", train_error_rate[k])
    print("Test Error Rate:", test_error_rate[k])
    #print("Coefficient Norm:", coefficient_norm[k])

min_error = np.min(test_error_rate)
opt_lambda_idx = np.argmin(test_error_rate)
opt_lambda = lambda_interval[opt_lambda_idx]
#print(w_est)
#print(y_train)
#print(y_train_est)
#print(y_test_est)
print("Optimal Lambda:",opt_lambda)


mu (mean of X_train): [4.9740931e-01 4.9243513e-01 9.7362695e+03 1.3914505e+00 1.1073053e+01
 4.8658550e+03 6.6647668e+02 1.2485492e+03]
sigma (std of X_train): [4.9999356e-01 1.1432965e-01 3.5229573e+03 3.2024434e-01 5.1639738e+00
 1.8886897e+03 4.0491800e+02 4.3628192e+02]

Lambda = 0.01 (C = 100.0):
Train Error Rate: 0.12953367875647667
Test Error Rate: 0.19047619047619047

Lambda = 0.010985411419875584 (C = 91.02981779915217):
Train Error Rate: 0.12953367875647667
Test Error Rate: 0.19047619047619047

Lambda = 0.012067926406393288 (C = 82.86427728546843):
Train Error Rate: 0.12953367875647667
Test Error Rate: 0.19047619047619047

Lambda = 0.013257113655901088 (C = 75.4312006335462):
Train Error Rate: 0.12953367875647667
Test Error Rate: 0.19047619047619047

Lambda = 0.014563484775012436 (C = 68.66488450043002):
Train Error Rate: 0.12953367875647667
Test Error Rate: 0.19047619047619047

Lambda = 0.015998587196060583 (C = 62.505519252739724):
Train Error Rate: 0.12953367875647667
Tes

In [ ]:
#plots for this?

plt.figure(figsize=(8, 8))
#plt.plot(np.log10(lambda_interval), train_error_rate*100)
#plt.plot(np.log10(lambda_interval), test_error_rate*100)
#plt.plot(np.log10(opt_lambda), min_error*100, 'o')

plt.semilogx(lambda_interval, train_error_rate * 100)
plt.semilogx(lambda_interval, test_error_rate * 100)
plt.semilogx(opt_lambda, min_error * 100, "o")
plt.text(
    1e-2,
    0,
    "Minimum test error: "
    + str(np.round(min_error * 100, 2))
    + " % at 1e"
    + str(np.round(np.log10(opt_lambda), 2)),
)
plt.xlabel("Regularization strength, $\log_{10}(\lambda)$")
plt.ylabel("Error rate (%)")
plt.title("Classification error")
plt.legend(["Training error", "Test error", "Test minimum"], loc="upper right")
plt.ylim([0, 25])
plt.grid()
plt.show()

plt.figure(figsize=(8, 8))
plt.semilogx(lambda_interval, coefficient_norm, "k")
plt.ylabel("L2 Norm")
plt.xlabel("Regularization strength, $\log_{10}(\lambda)$")
plt.title("Parameter vector L2 norm")
plt.grid()
plt.show()

KeyboardInterrupt: 

In [ ]:

def naive_classifier_binary(X, y):
    np.random.seed(2450)
    y = y.squeeze()
    
# Example usage with lambda and alpha ranges
#lambda_values = np.logspace(-5, 1, 10)  # Replace with desired range for logistic regression
#alpha_values = np.logspace(-5, 1, 10)   # Replace with desired range for naive classifier


    # Naive Bayes classifier parameters for binary features
    alpha = 1.0  # Laplace smoothing
    fit_prior = True  # Change if you want uniform prior
    
    # Set up K-fold cross-validation
    K = 10
    #CV = KFold(n_splits=K, shuffle=True)
    CV = model_selection.KFold(n_splits=K,shuffle=True)
    errors = np.zeros(K)
    train_errors = np.zeros(K)
    test_errors = np.zeros(K)
    k = 0

    
    for train_index, test_index in CV.split(X):
        print(f'Cross-validation fold: {k+1}/{K}')
        
        # Split data into training and test sets
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Initialize and fit BernoulliNB for binary classification
        nb_classifier = BernoulliNB(alpha=alpha, fit_prior=fit_prior)
        nb_classifier.fit(X_train, y_train)
        
         # Predictions for test and train sets
        y_test_est = nb_classifier.predict(X_test)
        y_train_est = nb_classifier.predict(X_train)

        # Calculate error rates
        test_errors[k] = np.mean(y_test_est != y_test)
        train_errors[k] = np.mean(y_train_est != y_train)
        
        # Predict and calculate error
        y_pred = nb_classifier.predict(X_test)
        errors[k] = np.mean(y_pred != y_test)
        print(y_pred)
        k += 1

    # Print mean error across folds
    print(f"Mean Test Error Rate: {100 * np.mean(errors):.2f}%")
     # Plot the classification error rate
    #print("Error rate: {0}%".format(100 * np.mean(errors))) 
    #print("Coefficient Norm:", coefficient_norm[k])
    print("Average Train Error Rate: {:.2f}%".format(100 * np.mean(train_errors)))
    print("Average Test Error Rate: {:.2f}%".format(100 * np.mean(test_errors)))

naive_classifier_binary(X, y)  # Call the function with appropriate arguments

Cross-validation fold: 1/10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Cross-validation fold: 2/10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Cross-validation fold: 3/10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Cross-validation fold: 4/10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Cross-validation fold: 5/10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Cross-validation fold: 6/10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Cross-validation fold: 7/10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Cross-validation fold: 8/10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Cross-validation fold: 9/10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Cross-validation fold: 10/10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Mean Test Error Rate: 41.15%
Average Train Error Rate: 41

In [140]:

def baseline_classification(y_train,y_test):  
    error_rates = []
    # Baseline
    K = 10
    CV = model_selection.KFold(n_splits=K, shuffle=True)

    # Find the most frequent class in the training labels
    unique_classes, counts = np.unique(y_train, return_counts=True)
    most_frequent_class = unique_classes[np.argmax(counts)]
    print("The most frequent class is:", most_frequent_class)

    #Create predictions for the test set using the most frequent class
    y_pred_baseline = np.full(y_test.shape, most_frequent_class)  # predict the most frequent class for all

    #Evaluate the baseline model
    baseline_accuracy = accuracy_score(y_test, y_pred_baseline)
    baseline_error = 1 - baseline_accuracy
    error_rates.append(baseline_error)

    # Calculate the average test error rate across all folds
    avg_test_error_rate = np.mean(error_rates)
    print(f"Average Test Error Rate: {avg_test_error_rate:.2f}")
    print(f"Baseline Accuracy: {baseline_accuracy:.2f}")
baseline_classification(y_train,y_test)

The most frequent class is: 1.0
Average Test Error Rate: 0.43
Baseline Accuracy: 0.57


In [ ]:
#2 level crossvalidation

def cross_validation(X, y, K1=10, K2=10, lambda_values=None, alpha_values=None):
    # Define outer and inner cross-validation objects
    outer_cv = KFold(n_splits=K1, shuffle=True, random_state=42)
    inner_cv = KFold(n_splits=K2, shuffle=True, random_state=42)

    results = []

    # Outer cross-validation loop
    for outer_fold, (train_idx, test_idx) in enumerate(outer_cv.split(X), start=1):
        X_train_outer, X_test_outer = X[train_idx], X[test_idx]
        y_train_outer, y_test_outer = y[train_idx], y[test_idx]

        best_lambda = None
        best_alpha = None
        lowest_error_logistic = float('inf')
        lowest_error_naive = float('inf')

        # Inner cross-validation loop for hyperparameter tuning
        for train_idx_inner, val_idx_inner in inner_cv.split(X_train_outer):
            X_train_inner, X_val_inner = X[train_idx_inner], X[train_idx_inner]
            y_train_inner, y_val_inner = y[train_idx_inner], y[val_idx_inner]

            # Logistic Regression - finding best lambda
            for lambda_ in lambda_values:
                error_logistic = logistic_regression(X_train_inner, y_train_inner, X_val_inner, y_val_inner, lambda_)
                if error_logistic < lowest_error_logistic:
                    lowest_error_logistic = error_logistic
                    best_lambda = lambda_

            # Naive Bayes - finding best alpha
            for alpha in alpha_values:
                error_naive = naive_classifier_binary(X_train_inner, y_train_inner, X_val_inner, y_val_inner, alpha)
                if error_naive < lowest_error_naive:
                    lowest_error_naive = error_naive
                    best_alpha = alpha

        # Evaluate on the outer test set with the best hyperparameters
        test_error_logistic = logistic_regression(X_train_outer, y_train_outer, X_test_outer, y_test_outer, best_lambda)
        test_error_naive = naive_classifier_binary(X_train_outer, y_train_outer, X_test_outer, y_test_outer, best_alpha)
        baseline_error = baseline_classification(y_train_outer, y_test_outer)

        results.append((outer_fold, best_lambda, test_error_logistic, best_alpha, test_error_naive, baseline_error))

    # Display results
    print(f"{'Fold':<6} {'Best Lambda':<12} {'Logistic Error':<15} {'Best Alpha':<10} {'Naive Error':<12} {'Baseline Error'}")
    for result in results:
        print(f"{result[0]:<6} {result[1]:<12} {result[2]:<15.4f} {result[3]:<10} {result[4]:<12.4f} {result[5]:.4f}")

    return results

# Example usage with lambda and alpha ranges
lambda_values = np.logspace(-5, 1, 10)  # Replace with desired range for logistic regression
alpha_values = np.logspace(-5, 1, 10)   # Replace with desired range for naive classifier

# Run cross-validation
results = cross_validation(X_train_tensor, y_train_tensor, lambda_values=lambda_values, alpha_values=alpha_values)

NameError: name 'X_train_tensor' is not defined

In [ ]:
# 2 Level crossvalidation
import numpy as np
#from data_import import *
from sklearn import model_selection
#from dataProcessing import *
#from utils import *

old_y = np.where( y != 0, np.ma.log10(y),0)
old_X = X
one_out_of_y = X[:,-4:]
y = one_out_of_k_to_numbers(one_out_of_y)
X = np.c_[old_X[:,:-4],old_y]

K1 = 10
K2 = 10

#model parameters
lambdas = np.logspace(-4, .5, num=100, endpoint=True, base=10.0)
hs = []#[1,2,3,5,10,20,30,50,100]
alphas = [0.1, 1.0, 10.0]  # Example values for alpha, you can adjust these
models = "logistic regression lambdas: " + np.array2string(np.array(lambdas)) + "; ANN hs: "+ np.array2string(np.array(hs))
with open("outputs/class/classification_models.txt","w") as file:
    file.write(models)
    file.close()
number_of_logreg_models = len(lambdas)
number_of_ANNs = len(hs)
#inner and outer cross-validation splits
outer_CV = model_selection.KFold(K1,shuffle=True)
inner_CV = model_selection.KFold(K2,shuffle=True)


outer_fold = 0
outer_error_array = np.zeros(K1)
selected_models = np.zeros(K1)
for outer_train_index, outer_test_index in outer_CV.split(X,y):
    outer_X_train = X[outer_train_index].astype('float64')
    outer_y_train = y[outer_train_index].astype('float64')
    outer_X_test = X[outer_test_index].astype('float64')
    outer_y_test = y[outer_test_index].astype('float64')
    #Inner cross-validation loop
    inner_error_matrix = np.zeros((K2,number_of_logreg_models+number_of_ANNs+1))
    inner_genE_array = np.zeros(number_of_logreg_models+number_of_ANNs+1)
    inner_fold = 0
    for inner_train_index, inner_test_index in inner_CV.split(outer_X_train,outer_y_train):
        inner_X_train = outer_X_train[inner_train_index]
        inner_y_train = outer_y_train[inner_train_index]
        inner_X_test = outer_X_train[inner_test_index]
        inner_y_test = outer_y_train[inner_test_index]
        
        for l in range(len(lambdas)):
            inner_error_matrix[inner_fold,l] = logistic_regression(inner_X_train,inner_y_train,inner_X_test,inner_y_test,lambdas[l])
            print("LogReg Nr. "+str(l)+" error: "+str( inner_error_matrix[inner_fold,l]))
        
        for h in range(len(hs)):
            inner_error_matrix[inner_fold,number_of_logreg_models+h] = ANN_gen_error(inner_X_train,inner_y_train,inner_X_test,inner_y_test,hs[h],"classification")
            print("ANN Nr. "+str(l)+" error: "+str( inner_error_matrix[inner_fold,number_of_logreg_models+h]))
        
        inner_error_matrix[inner_fold,-1] = baseline_classification_model(inner_y_train,inner_y_test)
        print("Baseline: "+str(inner_error_matrix[inner_fold,-1]))
        
        # Naive Bayes classifier errors with different alpha values
        for a, alpha in enumerate(alphas):
            inner_error_matrix[inner_fold, number_of_logreg_models + number_of_ann_models + a] = naive_classifier(inner_X_train, inner_y_train, inner_X_test, inner_y_test, alpha)
            print("Naive Bayes alpha=" + str(alpha) + " error: " + str(inner_error_matrix[inner_fold, number_of_logreg_models + number_of_ann_models + a]))

        
        print("inner fold "+str(inner_fold)+" done")
        inner_fold = inner_fold + 1
    
    np.savetxt("outputs/class/classification_OuterFold_"+str(outer_fold)+"_inner_error_matrix.csv",inner_error_matrix,delimiter=",",fmt='%1.10f')
    for index in range(len(inner_genE_array)):
        inner_genE_array[index] = inner_error_matrix.T[index].mean()
    model = np.where( inner_genE_array == inner_genE_array.min())[0][0]
    selected_models[outer_fold] = model # save the selected model
    if model in range(number_of_logreg_models):
        outer_error_array[outer_fold] = logistic_regression(outer_X_train,outer_y_train,outer_X_test,outer_y_test,lambdas[model])
    elif model in range(number_of_logreg_models,number_of_logreg_models+number_of_ANNs):
        outer_error_array[outer_fold] = ANN_gen_error(outer_X_train,outer_y_train,outer_X_test,outer_y_test,hs[model-number_of_logreg_models],"classification")
    else:
        outer_error_array[outer_fold] =baseline_classification_model(outer_y_train,outer_y_test)
    print("Gen error: "+str(outer_error_array[outer_fold]))
    print("outer Fold "+str(outer_fold) +" done")
    outer_fold = outer_fold + 1
np.savetxt("outputs/class/classification_outer_error_array.csv",outer_error_array,delimiter=",",fmt='%1.10f')
np.savetxt("outputs/class/classification_selected_models.csv",selected_models,delimiter=",",fmt='%1.10f')
print(outer_error_array)
print("Gen Error: " + str(outer_error_array.mean()))


NameError: name 'one_out_of_k_to_numbers' is not defined

In [ ]:
print("Fold | h* | Test Error")
for r in test_results:
    print(f"{r[0]} | {r[1]} | {r[2]}")

h = [r[1] for r in test_results]

# Plot the histogram for the frequency plot of h
plt.figure(figsize=(10, 6))
plt.hist(h, bins=20, edgecolor='black', alpha=0.7)
plt.xticks(range(int(min(h))-1, int(max(h))+2, 1))
plt.xlabel('h')
plt.ylabel('Frequency')
plt.title('Frequency Plot for Number h')
plt.show()

Fold | h* | Test Error


NameError: name 'test_results' is not defined

In [ ]:
#statistical evaluation of your three models, McNemera’s test described in Box 11.3.2

from dtuimldmtools import mcnemar

# Compute the Jeffreys interval
alpha = 0.05
[thetahat, CI, p] = mcnemar(y_true, yhat[:, 0], yhat[:, 1], alpha=alpha)

print("theta = theta_A-theta_B point estimate", thetahat, " CI: ", CI, "p-value", p)

NameError: name 'y_true' is not defined

### Feature Selection

Use the features identified as significant from the PCA analysis. 

This includes 'modulus of rupture', 'shear parallel to grain', 'compression parallel to grain', 'modulus of elasticity', 'work to maximum load', and 'specific gravity'. 

These are strong candidates for your predictive models.

### Old stuff/ Inspiration

To predict a continuous variable based on other variables. 

To predict the values/property of one feature based on other features.

In [ ]:
# ANN Model

# Define variables
K1 = 10
K2 = 10

class ANN(nn.Module):
    def __init__(self, input_dim, h):
        super(ANN, self).__init__()
        self.layer1 = nn.Linear(input_dim, h)
        self.layer2 = nn.Linear(h,h)
        self.output_layer = nn.Linear(h, 1)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.output_layer(x)
        return x
    

# Cross-validation
outer_cv = KFold(n_splits=K1, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=K2, shuffle=True, random_state=42)

test_results =[]
inner_errors = []

# Outer Loop
for i, (train_idx, test_idx) in enumerate(outer_cv.split(X_tensor)):
    X_train, X_test = X_tensor[train_idx], X_tensor[test_idx]
    y_train, y_test = y_tensor[train_idx], y_tensor[test_idx]

    best_h = None
    #best_lambda = None
    best_model = None
    lowest_error = float('inf')

    # Inner Loop
    for h in range(1,20): # onyl try up to 20 hidden neurons
        inner_errors_for_fold = []

        for train_idx_in, val_idx in inner_cv.split(X_train):
            X_train_inner, X_val = X_train[train_idx_in], X_train[val_idx]
            y_train_inner, y_val = y_train[train_idx_in], y_train[val_idx]

            # Define model based on h  
            model = ANN(X_train_inner.shape[1], h)
            optimizer = optim.Adam(model.parameters(), lr=0.01)
            criterion = nn.MSELoss()

            # Training phase
            for epoch in range(100):
                model.train()
                optimizer.zero_grad()
                outputs = model(X_train_inner)
                loss = criterion(outputs, y_train_inner)
                loss.backward()
                optimizer.step()
            

            # Evaluation phase
            model.eval()
            with torch.no_grad():
                val_outputs = model(X_val)
                #val_loss = criterion(val_outputs, y_val)
                val_error = mean_squared_error(y_val.numpy(), val_outputs.numpy())
                inner_errors_for_fold.append(val_error)
            
        
        # Find best h for this fold
        avg_inner_error = np.mean(inner_errors_for_fold)
        if avg_inner_error < lowest_error: 
            lowest_error = avg_inner_error
            best_h = h
            
                

    # Evaluate the best model based on best h
    best_model = ANN(X_train.shape[1], best_h)
    optimizer = optim.Adam(best_model.parameters(), lr=0.01)
    criterion = nn.MSELoss()
    for epoch in range(50):
        best_model.train()
        optimizer.zero_grad()
        outputs = best_model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    best_model.eval()
    with torch.no_grad():
        test_outputs = best_model(X_test)
        test_error = mean_squared_error(y_test.numpy(), test_outputs.numpy())

    test_results.append((i+1, best_h, test_error))

In [ ]:
def baseline_classification_model(y_train,y_test):
    classifiers,counts = np.unique(y_train,return_counts=True)
    classifier = classifiers[np.where(counts==counts.max())][0]
    print(classifier)
    return np.count_nonzero(classifier==y_test)/len(y_test)
baseline_classification_model(y_train,y_test)

In [ ]:
#  Naive Classifier
from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import OneHotEncoder

#def naive_classifier(X, y):
#def naive_classifier(X_train,y_train,X_test,y_test):
np.random.seed(2450)
y = y.squeeze()
    #0
    # Naive Bayes classifier parameters
alpha = 1.0  # pseudo-count, additive parameter (Laplace correction if 1.0 or Lidtstone smoothing otherwise)
fit_prior = True  # uniform prior (change to True to estimate prior from data)
    # K-fold crossvalidation
K = 10
CV = model_selection.KFold(n_splits=K, shuffle=True)

    #X = X[:, 0:4]  # using all 4 letters. Not needed here, because we already have defined that on top?
    #X = OneHotEncoder().fit_transform(X=X)

errors = np.zeros(K)
train_errors = np.zeros(K)
test_errors = np.zeros(K)
k = 0
for train_index, test_index in CV.split(X):
    print('Crossvalidation fold: {0}/{1}'.format(k+1,K))

    # extract training and test set for current CV fold
    X_train = X[train_index, :]
    y_train = y[train_index]
    X_test = X[test_index, :]
    y_test = y[test_index]

    nb_classifier = MultinomialNB(alpha=alpha, fit_prior=fit_prior)
    nb_classifier.fit(X_train, y_train)
    y_est_prob = nb_classifier.predict_proba(X_test)
    y_est = np.argmax(y_est_prob, 1)

    # Predictions for test and train sets
    y_test_est = nb_classifier.predict(X_test)
    y_train_est = nb_classifier.predict(X_train)

    # Calculate error rates
    test_errors[k] = np.mean(y_test_est != y_test)
    train_errors[k] = np.mean(y_train_est != y_train)

    errors[k] = np.sum(y_est != y_test, dtype=float) / y_test.shape[0]
    k += 1

    # Plot the classification error rate
    #print("Error rate: {0}%".format(100 * np.mean(errors))) 
    #print("Coefficient Norm:", coefficient_norm[k])
    print("Average Train Error Rate: {:.2f}%".format(100 * np.mean(train_errors)))
    print("Average Test Error Rate: {:.2f}%".format(100 * np.mean(test_errors)))
    
#naive_classifier(X_train,y_train,X_test,y_test)
#naive_classifier(X_train,y_train,X_test,y_test)

In [ ]:
cwd = os.getcwd()
data = pd.read_csv(cwd + '/usa_wood_data_formatted.csv')
#data = pd.read_csv('C:/Users/ongji/OneDrive/Documents/DTU/02450 Introduction to Machine Learning and Data Mining/02450-Project-1/usa_wood_data_formatted.csv')
# print(data.head())  

# Clean data by removing columns with a significant amount of missing values
removed_columns = ['side_hardness','tension_perpendicular_to_grain','impact_bending']
data_cleaned = data.drop(columns=removed_columns)
data_cleaned = data_cleaned.dropna()
# print(data_cleaned.isnull().sum()) # 0 means data is clean


# Extract attribute names (1st row, column 0 to 13)
attributeNames = data_cleaned.columns[:13]
for i in range(len(attributeNames)):
    print(i, attributeNames[i])

# Extract class names to python list, then encode with integers (dict)
classLabels = data_cleaned['classification'].values
classNames = sorted(set(classLabels))
classDict = {className: index for index, className in enumerate(classNames)}
y = np.array([classDict[label] for label in classLabels])
print(y)

# Extract class names to python list, then encode with integers (dict)
classLabels = data_cleaned['moisture'].values
classNames = sorted(set(classLabels))
classDict = {className: index for index, className in enumerate(classNames)}
y = np.array([classDict[label] for label in classLabels])
print(y)


# Preallocate memory, then extract excel data to matrix X
X = data_cleaned.iloc[:,0:13].values

# Compute values of N, M and C.
N = len(y)
M = len(attributeNames)
C = len(classNames)

# Filter out numerical values
X_num = data_cleaned.iloc[:,6:].values

# Scale and substract mean from data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_num)
Y = X_scaled - np.ones((N, 1)) * X_scaled.mean(axis=0)

# PCA by computing SVD of Y
U, S, Vh = svd(Y, full_matrices=False)

# Compute variance explained by principal components
rho = (S * S) / (S * S).sum()

# print("Singular values:", S)
# print("Variance explained:", rho)
# print("Cumulative variance explained:", np.cumsum(rho))

# Convert arra
w = X_scaled[:,1:]
X_tensor = torch.FloatTensor(w)
#y_tensor = torch.FloatTensor(X_scaled[:,6])
y_tensor = torch.FloatTensor(X_scaled[:,1]).unsqueeze(1)
print("Xtensor:", X_tensor)
print("ytensor:",y_tensor)